In [20]:
# téléchargement

from pymatgen.symmetry import analyzer as az
from pymatgen.ext.matproj import MPRester
import numpy as np
from IPython.display import Image
from pymatgen.symmetry.analyzer import *
from pymatgen.io.cif import *
from pymatgen.core.operations import *
from pymatgen.symmetry.site_symmetries import *
import matplotlib.pyplot as plt
import matplotlib.image as img


with MPRester(cle_utilisateur) as m:
    structure = m.get_structure_by_material_id(cle_materiau)
    structure = az.SpacegroupAnalyzer(structure).get_primitive_standard_structure()
    

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
# paramètres : je les ai réunis ici afin d'avoir un code 'modulaire' : on peut choisir ce qu'on veut
# observer en fonction de paramètres listés ici

# pour le choix du matériau
cle_materiau = "mp-28554"

# propre à l'utilisateur
cle_utilisateur = "rEVEIFsc61iExKZbJ2QyepniwSgJ1m6W"

# le nombre de décimales qu'on veut voir pour les valeurs chiffrées
dec = 4

# 3 éléments de symétrie différents

In [22]:
def give_coord(A):
    """
    détermine les nouvelles coordonnées d'un point (x,y,z)
    en fonction de la matrice de rotation A:
    
    A_ij appartient a {1,0,-1}
    
                    1 0 0
    résultat si A = 0 1 0  : ['x','y','z']
                    0 0 1
    """
    x = ['x','y','z']
    y = ['']*3
    index = [0,1,2] 
    for i in index:
        for j in index:
            if A[i][j] == 1:
                if y[j] == '':
                    y[j] = x[i]
                else :
                    y[j] += '+'+x[i]
                
            if A[i][j] == -1:
                y[j] += '-'+ x[i]
    return y


In [23]:
an = az.SpacegroupAnalyzer(structure)
structure = an.get_conventional_standard_structure()
an = az.SpacegroupAnalyzer(structure)


# on récupère les informations de site sur 3 atomes différents
all_sites = structure.sites
sites = [all_sites[1], all_sites[6], all_sites[9]]

print("\nPositions des 3 atomes différents\n")

for site in sites:
    print('position de l\'atome "'+str(site.specie)+'" : '+str(np.around(site.frac_coords,dec)))
    

# on prend 3 opérations de symétries différentes de l'identité et qui ne sont pas l'opposé d'une opération déjà utilisée
all_symetries = an.get_symmetry_operations()
symetries = [all_symetries[7], all_symetries[12], all_symetries[25]]


# on fait un bel affichage
i = 0
for s in symetries :
    i+=1 
    rot_matrix = s.rotation_matrix
    tr_vector = s.translation_vector
    
    print('\n\nsymétrie '+str(i)+':'+"\n")
    print('matrice de rotation :')
    print(np.around(rot_matrix),end='\n\n')
    print(give_coord(rot_matrix))
    print('\nvecteur de translation : '+str(np.around(tr_vector,dec))+'\n')
    
    for site in sites :
        new_coord = np.dot(site.frac_coords, rot_matrix)+tr_vector
        




Positions des 3 atomes différents

position de l'atome "Ca" : [0.     0.     0.2313]
position de l'atome "As" : [0.3333 0.6667 0.1667]
position de l'atome "I" : [0. 0. 0.]


symétrie 1:

matrice de rotation :
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]

['-y', '-x', 'z']

vecteur de translation : [0. 0. 0.]



symétrie 2:

matrice de rotation :
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

['x', 'y', 'z']

vecteur de translation : [0.6667 0.3333 0.3333]



symétrie 3:

matrice de rotation :
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]

['-x', '-y', '-z']

vecteur de translation : [0.3333 0.6667 0.6667]



# Modélisation de la structure du matériaux 

## Structure initial

In [24]:
from jupyter_jsmol import JsmolView
from ipywidgets import Layout, widgets, interact
from pymatgen.core.structure import Structure
structure_transformed = structure.copy()
for site, sym_op in zip(sites, symetries):
    new_pos = sym_op.operate(site.frac_coords)
    structure_transformed.append(site.specie, new_pos, coords_are_cartesian=False)
CifWriter(structure_transformed).write_file("transformed_structure.cif")


In [25]:
view1 = JsmolView.from_file("mp-28554.cif", inline=True) 
view1

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [26]:
view2 = JsmolView.from_file("transformed_structure.cif", inline=True)
view2


JsmolView(layout=Layout(align_self='stretch', height='400px'))